In [1]:
from datetime import datetime
from pydantic import BaseModel, Field, ConfigDict
from typing import List, Optional
from schema import MetservicePointTimeRequest, GPTResponseToWeatherQuery, GPTMessage
from utils.constants import METSERVICE_VARIABLES
from loguru import logger
import instructor
import openai
import googlemaps
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

pydantic_client = instructor.apatch(
    openai.AsyncOpenAI(api_key=os.environ['OPENAI_API_KEY']))
client = openai.AsyncOpenAI(api_key=os.environ['OPENAI_API_KEY'])
gmaps = googlemaps.Client(key=os.environ['GOOGLE_MAPS_API_KEY'])

/var/folders/0s/pn0r0b8120ngcq1d6j4d6tzc0000gn/T/ipykernel_16009/1754544606.py:3: DeprecationWarning: apatch is deprecated, use patch instead
  pydantic_client = instructor.apatch(


In [3]:
data_store = []
chat_log = []

In [4]:
response_model = GPTResponseToWeatherQuery
user_message = "What's the weather like in Auckland tomorrow?"
chat_log.append(GPTMessage(role="user", content=user_message))
GPT_messages = [
    {"role": "system", "content":
             f"""You are WeatherBot, a chatbot that can answer any user query about the weather.

            You will receive a user query and must respond with the outlined JSON structure.
            You'll also receive historical chat logs so you have context on the conversation.
            The date and time right now is {datetime.now().isoformat()}. This might be helpful if the user query contains a relative date or time.

            When answering, please adhere to the following rules:
            - If the query is not weather related, please answer the user query as best as you can and remind them that you are a weather bot.
            - You can only answer user queries based on the information you receive from a weather API.
            - Previous information from the weather API can be found in the provided DATA STORE.
            - If the data is insufficient to answer the query or empty, you can request additional data from the API by returning sufficient_data_check as False and request additional data using the other fields.
            - If the data is insufficient, include a holding message in the response field while we request data from the API.
            - Please ensure that your request will provide the right data for another language model to answer the query.
            - For variables, you can select from the provided list. Please only select the most relevant variables to answer the query.
            - Make sure you get enough data. For example, if the request is for tomorrow, get data for the whole day.
            - If weather_query_check and sufficient_data_check are both true, you must respond to the weather query as WeatherBot. Open your query with a succint summary of the weather and give the most important details.
            - If the request doesn't contain a specific time, make sure that the from_datetime, interval and repeat fields would request enough data from the API for a language model to answer the query with the received data.
            - The from_datetime field can only be values on the hour, e.g. 2022-01-01T00:00:00Z, 2022-01-01T01:00:00Z, 2022-01-01T02:00:00Z, etc.
            - If the user request is for the weather now, please request the data at the previous hour mark plus one additional hour of data.
            - Give any temperature in degrees Celsius, any wind speed in kilometres per hour, and any time in NZDT.
            - Don't go into too much detail, be succinct. No one likes to hear every single detail about the weather.

            ______________________________
            DATA STORE
            ______________________________
            {data_store}
            ______________________________
            VARIABLES
            ______________________________
            {METSERVICE_VARIABLES}
            REMEMBER THE RULES:
            - If the query is not weather related, please answer the user query as best as you can and remind them that you are a weather bot.
            - You can only answer user queries based on the information you receive from a weather API.
            - Previous information from the weather API can be found in the provided DATA STORE.
            - If the data is insufficient to answer the query or empty, you can request additional data from the API by returning sufficient_data_check as False and request additional data using the other fields.
            - If the data is insufficient, include a holding message in the response field while we request data from the API.
            - Please ensure that your request will provide the right data for another language model to answer the query.
            - For variables, you can select from the provided list. Please only select the most relevant variables to answer the query.
            - Make sure you get enough data. For example, if the request is for tomorrow, get data for the whole day.
            - If weather_query_check and sufficient_data_check are both true, you must respond to the weather query as WeatherBot. Open your query with a succint summary of the weather and give the most important details.
            - If the request doesn't contain a specific time, make sure that the from_datetime, interval and repeat fields would request enough data from the API for a language model to answer the query with the received data.
            - The from_datetime field can only be values on the hour, e.g. 2022-01-01T00:00:00Z, 2022-01-01T01:00:00Z, 2022-01-01T02:00:00Z, etc.
            - If the user request is for the weather now, please request the data at the previous hour mark plus one additional hour of data.
            - Give any temperature in degrees Celsius, any wind speed in kilometres per hour, and any time in NZDT.
            - Don't go into too much detail, be succinct. No one likes to hear every single detail about the weather."""
     },
]

In [5]:
for message in chat_log:
    print(message)
    GPT_messages.append({"role": message.role, "content": message.content})
print(f"Messages: {GPT_messages}")

role='user' content="What's the weather like in Auckland tomorrow?"
Messages: [{'role': 'system', 'content': "You are WeatherBot, a chatbot that can answer any user query about the weather.\n\n            You will receive a user query and must respond with the outlined JSON structure.\n            You'll also receive historical chat logs so you have context on the conversation.\n            The date and time right now is 2024-02-11T16:52:24.353598. This might be helpful if the user query contains a relative date or time.\n\n            When answering, please adhere to the following rules:\n            - If the query is not weather related, please answer the user query as best as you can and remind them that you are a weather bot.\n            - You can only answer user queries based on the information you receive from a weather API.\n            - Previous information from the weather API can be found in the provided DATA STORE.\n            - If the data is insufficient to answer the 

In [6]:
response = await pydantic_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    response_model=response_model,
    messages=GPT_messages,
)

In [7]:
class Message(BaseModel):
    text: str
    name: str
    stamp: str
    avatar: str
    sent: bool

In [8]:
import httpx
import json

In [9]:
async def GPT_response(user_message: str, chat_log: list[GPTMessage], data_store: list[str], response_model: BaseModel) -> GPTResponseToWeatherQuery:
    GPT_messages = [
        {"role": "system", "content":
         f"""You are WeatherBot, a chatbot that can answer any user query about the weather.

            You will receive a user query and must respond with the outlined JSON structure.
            You'll also receive historical chat logs so you have context on the conversation.
            The date and time right now is {datetime.now().isoformat()}. This might be helpful if the user query contains a relative date or time.

            When answering, please adhere to the following rules:
            - If the query is not weather related, please answer the user query as best as you can and remind them that you are a weather bot.
            - You can only answer user queries based on the information you receive from a weather API.
            - Previous information from the weather API can be found in the provided DATA STORE.
            - If the data is insufficient to answer the query or empty, you can request additional data from the API by returning sufficient_data_check as False and request additional data using the other fields.
            - If the data is insufficient, include a holding message in the response field while we request data from the API.
            - Please ensure that your request will provide the right data for another language model to answer the query.
            - For variables, you can select from the provided list. Please only select the most relevant variables to answer the query.
            - Make sure you get enough data. For example, if the request is for tomorrow, get data for the whole day.
            - If weather_query_check and sufficient_data_check are both true, you must respond to the weather query as WeatherBot. Open your query with a succint summary of the weather and give the most important details.
            - If the request doesn't contain a specific time, make sure that the from_datetime, interval and repeat fields would request enough data from the API for a language model to answer the query with the received data.
            - The from_datetime field can only be values on the hour, e.g. 2022-01-01T00:00:00Z, 2022-01-01T01:00:00Z, 2022-01-01T02:00:00Z, etc.
            - If the user request is for the weather now, please request the data at the previous hour mark plus one additional hour of data.
            - Give any temperature in degrees Celsius, any wind speed in kilometres per hour, and any time in NZDT.
            - Don't go into too much detail, be succinct. No one likes to hear every single detail about the weather.

            ______________________________
            DATA STORE
            ______________________________
            {data_store}
            ______________________________
            VARIABLES
            ______________________________
            {METSERVICE_VARIABLES}

            REMEMBER THE RULES:
            - If the query is not weather related, please answer the user query as best as you can and remind them that you are a weather bot.
            - You can only answer user queries based on the information you receive from a weather API.
            - Previous information from the weather API can be found in the provided DATA STORE.
            - If the data is insufficient to answer the query or empty, you can request additional data from the API by returning sufficient_data_check as False and request additional data using the other fields.
            - If the data is insufficient, include a holding message in the response field while we request data from the API.
            - Please ensure that your request will provide the right data for another language model to answer the query.
            - For variables, you can select from the provided list. Please only select the most relevant variables to answer the query.
            - Make sure you get enough data. For example, if the request is for tomorrow, get data for the whole day.
            - If weather_query_check and sufficient_data_check are both true, you must respond to the weather query as WeatherBot. Open your query with a succint summary of the weather and give the most important details.
            - If the request doesn't contain a specific time, make sure that the from_datetime, interval and repeat fields would request enough data from the API for a language model to answer the query with the received data.
            - The from_datetime field can only be values on the hour, e.g. 2022-01-01T00:00:00Z, 2022-01-01T01:00:00Z, 2022-01-01T02:00:00Z, etc.
            - If the user request is for the weather now, please request the data at the previous hour mark plus one additional hour of data.
            - Give any temperature in degrees Celsius, any wind speed in kilometres per hour, and any time in NZDT.
            - Don't go into too much detail, be succinct. No one likes to hear every single detail about the weather."""
         },
    ]
    for message in chat_log:
        print(message)
        GPT_messages.insert(-1, {"role": message.role, "content": message.content})
    logger.info(f"Messages: {GPT_messages}")
    response = await pydantic_client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=response_model,
        messages=GPT_messages,
    )

    assert isinstance(response, GPTResponseToWeatherQuery)
    logger.info(f"GPT response: {response}")
    return response

In [10]:
async def location_to_lat_lon(response: GPTResponseToWeatherQuery) -> MetservicePointTimeRequest:
    logger.info(f"Location: {response.location}")
    async with httpx.AsyncClient() as request_client:
        geocode_response = await request_client.get(
            "https://maps.googleapis.com/maps/api/geocode/json",
            params={
                "address": response.location,
                "key": os.environ["GOOGLE_MAPS_API_KEY"]
            }
        )
    if geocode_response.status_code != 200:
        raise ValueError(f"Request failed with status code {geocode_response.status_code}")
    geocode_response = geocode_response.json()

    metservice_request = MetservicePointTimeRequest(
        latitude=geocode_response['results'][0]['geometry']['location']['lat'],
        longitude=geocode_response['results'][0]['geometry']['location']['lng'],
        variables=response.variables,
        from_datetime=response.from_datetime,
        interval=response.interval,
        repeat=response.repeat
    )

    return metservice_request

In [11]:
async def metservice_api_call(request: MetservicePointTimeRequest) -> str:
    logger.info(f"Request: {request}")
    async with httpx.AsyncClient() as request_client:
        response = await request_client.post(
            "https://forecast-v2.metoceanapi.com/point/time",
            headers={"x-api-key": os.environ["METSERVICE_KEY"]},
            json={
                "points": [{
                    "lon": request.longitude,
                    "lat": request.latitude,
                }],
                "variables": request.variables,
                "time": {
                    "from": request.from_datetime,
                    "interval": request.interval,
                    "repeat": request.repeat
                }
            }
        )

    if response.status_code != 200:
        raise ValueError(f"Request failed with status code {response.status_code}")
    logger.info(f"API response: {response.json()}")
    return json.dumps(response.json(), indent=4)

In [12]:
async def generate_response(data: str, user_message: str) -> str:
    logger.info(f"Data: {data}")
    response = await client.chat.completions.create(
        model="gpt-4-turbo-preview",
        # stream=True,
        messages=[
            {"role": "system", "content":
             f"""You are an expert weatherperson, skilled at answering requests about the weather in a clear, helpful and succinct manner. Use your expertise to answer the user query, only making use of the weather data below, which is being supplied to you by a weather API based on the user request.
        ______________________________
        DATA
        ______________________________
        {data}"""
            },
            {"role": "user", "content": user_message}
        ]
    )
    logger.info(f"Response: {response.choices[0].message.content}")
    return response.choices[0].message.content

In [13]:
messages =[]

In [14]:
messages.append(user_message)

In [15]:
bot_stamp = datetime.now().strftime("%H:%M")
messages.append(Message(text=response.response, name="Bot", stamp=bot_stamp, avatar="", sent=False))
chat_log.append(GPTMessage(role="assistant", content=response.response))

if response.weather_query_check and not response.sufficient_data_check:
    chat_log.append(GPTMessage(role="user", content=user_message))
    metservice_query = await location_to_lat_lon(response)
    data = await metservice_api_call(metservice_query)
    data_store.append(data)
    reply = await GPT_response(user_message, chat_log, data_store, GPTResponseToWeatherQuery)
    bot_stamp = datetime.now().strftime("%H:%M")
    messages.append(Message(text=reply.response, name="Bot", stamp=bot_stamp, avatar="", sent=False))
    chat_log.append(GPTMessage(role="assistant", content=reply.response))

2024-02-11 16:52:28.451 | INFO     | __main__:location_to_lat_lon:2 - Location: Auckland
2024-02-11 16:52:28.832 | INFO     | __main__:metservice_api_call:2 - Request: latitude=-36.85088270000001 longitude=174.7644881 variables=['air.temperature.at-2m', 'cloud.cover', 'precipitation.rate', 'wind.speed.at-10m'] from_datetime='2024-02-12T00:00:00Z' interval='1h' repeat=24
2024-02-11 16:52:29.200 | INFO     | __main__:metservice_api_call:23 - API response: {'dimensions': {'point': {'type': 'point', 'units': 'unknown', 'data': [{'lon': 174.7644881, 'lat': -36.85088270000001}]}, 'time': {'type': 'time', 'units': 'unknown', 'data': ['2024-02-12T00:00:00Z', '2024-02-12T01:00:00Z', '2024-02-12T02:00:00Z', '2024-02-12T03:00:00Z', '2024-02-12T04:00:00Z', '2024-02-12T05:00:00Z', '2024-02-12T06:00:00Z', '2024-02-12T07:00:00Z', '2024-02-12T08:00:00Z', '2024-02-12T09:00:00Z', '2024-02-12T10:00:00Z', '2024-02-12T11:00:00Z', '2024-02-12T12:00:00Z', '2024-02-12T13:00:00Z', '2024-02-12T14:00:00Z', '2024

role='user' content="What's the weather like in Auckland tomorrow?"
role='assistant' content='I need to gather data on the weather in Auckland tomorrow before I can provide an accurate forecast. Please hold on while I request the necessary information.'
role='user' content="What's the weather like in Auckland tomorrow?"


2024-02-11 16:52:34.082 | INFO     | __main__:GPT_response:63 - GPT response: response="I'm checking the weather forecast for Auckland tomorrow. Please wait a moment." weather_query_check=True sufficient_data_check=True data_check_rationale=None location='Auckland' variables=['air.temperature.at-2m', 'cloud.cover', 'precipitation.rate', 'wind.speed.at-10m'] from_datetime='2024-02-12T00:00:00Z' interval='1h' repeat=24


In [16]:
print(messages)

["What's the weather like in Auckland tomorrow?", Message(text='I need to gather data on the weather in Auckland tomorrow before I can provide an accurate forecast. Please hold on while I request the necessary information.', name='Bot', stamp='16:52', avatar='', sent=False), Message(text="I'm checking the weather forecast for Auckland tomorrow. Please wait a moment.", name='Bot', stamp='16:52', avatar='', sent=False)]


In [17]:
print(GPT_messages)

[{'role': 'system', 'content': "You are WeatherBot, a chatbot that can answer any user query about the weather.\n\n            You will receive a user query and must respond with the outlined JSON structure.\n            You'll also receive historical chat logs so you have context on the conversation.\n            The date and time right now is 2024-02-11T16:52:24.353598. This might be helpful if the user query contains a relative date or time.\n\n            When answering, please adhere to the following rules:\n            - If the query is not weather related, please answer the user query as best as you can and remind them that you are a weather bot.\n            - You can only answer user queries based on the information you receive from a weather API.\n            - Previous information from the weather API can be found in the provided DATA STORE.\n            - If the data is insufficient to answer the query or empty, you can request additional data from the API by returning suff

In [18]:
print(chat_log)

[GPTMessage(role='user', content="What's the weather like in Auckland tomorrow?"), GPTMessage(role='assistant', content='I need to gather data on the weather in Auckland tomorrow before I can provide an accurate forecast. Please hold on while I request the necessary information.'), GPTMessage(role='user', content="What's the weather like in Auckland tomorrow?"), GPTMessage(role='assistant', content="I'm checking the weather forecast for Auckland tomorrow. Please wait a moment.")]


In [19]:
print(data_store)

['{\n    "dimensions": {\n        "point": {\n            "type": "point",\n            "units": "unknown",\n            "data": [\n                {\n                    "lon": 174.7644881,\n                    "lat": -36.85088270000001\n                }\n            ]\n        },\n        "time": {\n            "type": "time",\n            "units": "unknown",\n            "data": [\n                "2024-02-12T00:00:00Z",\n                "2024-02-12T01:00:00Z",\n                "2024-02-12T02:00:00Z",\n                "2024-02-12T03:00:00Z",\n                "2024-02-12T04:00:00Z",\n                "2024-02-12T05:00:00Z",\n                "2024-02-12T06:00:00Z",\n                "2024-02-12T07:00:00Z",\n                "2024-02-12T08:00:00Z",\n                "2024-02-12T09:00:00Z",\n                "2024-02-12T10:00:00Z",\n                "2024-02-12T11:00:00Z",\n                "2024-02-12T12:00:00Z",\n                "2024-02-12T13:00:00Z",\n                "2024-02-12T14:00:00Z